<a href="https://colab.research.google.com/github/Danicoder/Especializacion_IA_BigData/blob/main/PipeLine_Y_transformadores_personalizados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#EJERCICIO PRACTICO DE PIPELINES Y TRANSFORMADORES

In [1]:
pip install liac-arff

  Preparing metadata (setup.py) ... done
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11716 sha256=1072155f6fbc598ab41ebebcd283791988d1f38416ee44665e157d9787d3bc19
  Stored in directory: /root/.cache/pip/wheels/00/23/31/5e562fce1f95aabe57f2a7320d07433ba1cd152bcde2f6a002
Successfully built liac-arff


In [69]:
import arff
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

In [7]:
def load_kdd_dataset(data_path):
    """Lectura del conjunto de datos NSL-KDD."""
    with open(data_path, 'r') as train_set:
        dataset = arff.load(train_set)
        attributes = [attr[0] for attr in dataset["attributes"]]
        return pd.DataFrame(dataset["data"], columns=attributes)

In [8]:
def train_val_test_split(df, rstate=42, shuffle=True,stratify=None):
    strat = df[stratify] if stratify else None
    train_set, test_set = train_test_split(
    df, test_size=0.4, random_state=rstate,
    shuffle=shuffle, stratify=strat)
    strat = test_set[stratify] if stratify else None
    val_set, test_set = train_test_split(
    test_set, test_size=0.5, random_state=rstate,
    shuffle=shuffle, stratify=strat)
    return (train_set, val_set, test_set)

In [11]:
from google.colab import files

# Subir el archivo manualmente
uploaded = files.upload()

Saving KDDTrain%2B.arff to KDDTrain%2B.arff


In [13]:
df = load_kdd_dataset("KDDTrain%2B.arff")
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0.0,tcp,ftp_data,SF,491.0,0.0,0,0.0,0.0,0.0,...,25.0,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0.0,udp,other,SF,146.0,0.0,0,0.0,0.0,0.0,...,1.0,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,26.0,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,anomaly
3,0.0,tcp,http,SF,232.0,8153.0,0,0.0,0.0,0.0,...,255.0,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0.0,tcp,http,SF,199.0,420.0,0,0.0,0.0,0.0,...,255.0,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80175,0.0,tcp,smtp,SF,1361.0,328.0,0,0.0,0.0,0.0,...,170.0,0.95,0.03,0.02,0.01,0.00,0.00,0.00,0.00,normal
80176,0.0,tcp,http,SF,212.0,188.0,0,0.0,0.0,0.0,...,255.0,1.00,0.00,0.03,0.02,0.00,0.00,0.00,0.00,normal
80177,0.0,icmp,ecr_i,SF,1032.0,0.0,0,0.0,0.0,0.0,...,182.0,0.71,0.01,0.71,0.00,0.00,0.00,0.00,0.00,anomaly
80178,0.0,tcp,smtp,SF,1833.0,366.0,0,0.0,0.0,0.0,...,180.0,0.88,0.08,0.02,0.01,0.00,0.00,0.00,0.00,normal


In [15]:
train_set, val_set, test_set = train_val_test_split(df,
stratify='protocol_type') #mantiene equilibrado los tres subconjuntos de datos.

In [20]:
print(f"Tamaño del conjunto de entrenamiento: {len(train_set)}")
print(f"Tamaño del conjunto de validación: {len(val_set)}")
print(f"Tamaño del conjunto de prueba: {len(test_set)}")

Tamaño del conjunto de entrenamiento: 48108
Tamaño del conjunto de validación: 16036
Tamaño del conjunto de prueba: 16036


In [21]:
X_train = train_set.drop("class", axis=1)
y_train = train_set["class"].copy()

In [23]:
# Añadir NaN según las condiciones especificadas
df.loc[(df['src_bytes'] > 400) & (df['dst_bytes'] < 800), 'src_bytes'] = np.nan
df.loc[(df['dst_bytes'] > 500) & (df['dst_bytes'] < 200), 'src_bytes'] = np.nan

In [24]:
X_train

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
23846,0.0,udp,domain_u,SF,46.0,46.0,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00
29862,0.0,tcp,sql_net,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,5.0,0.02,0.07,0.00,0.00,1.0,1.0,0.00,0.00
57090,0.0,tcp,http_443,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,12.0,0.05,0.05,0.00,0.00,1.0,1.0,0.00,0.00
59386,0.0,tcp,http,SF,338.0,1158.0,0,0.0,0.0,0.0,...,29.0,255.0,1.00,0.00,0.03,0.05,0.0,0.0,0.00,0.00
29823,0.0,tcp,whois,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,3.0,0.01,0.05,0.00,0.00,1.0,1.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50433,0.0,tcp,http,SF,313.0,1086.0,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00
18596,0.0,tcp,http,SF,333.0,864.0,0,0.0,0.0,0.0,...,79.0,255.0,1.00,0.00,0.01,0.07,0.0,0.0,0.78,0.85
24293,0.0,udp,private,SF,1.0,0.0,0,0.0,0.0,0.0,...,255.0,19.0,0.07,0.23,1.00,0.00,0.0,0.0,0.00,0.00
64492,0.0,tcp,http,SF,210.0,299.0,0,0.0,0.0,0.0,...,29.0,255.0,1.00,0.00,0.03,0.01,0.0,0.0,0.00,0.00


#Borrado de valores nulos

In [34]:
from sklearn.base import BaseEstimator, TransformerMixin

class DeleteNanRows(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None): # para construir el transformador
        return self
    def transform(self, X, y=None): # devolvemos el conjunto de datos pero eliminando las filas
        return X.dropna()

In [35]:
delete_nan = DeleteNanRows()#almaceno las filas que tienen nan

In [37]:
X_train_prep = delete_nan.fit_transform(X_train)#para obtener los transformers
X_train_prep

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
23846,0.0,udp,domain_u,SF,46.0,46.0,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00
29862,0.0,tcp,sql_net,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,5.0,0.02,0.07,0.00,0.00,1.0,1.0,0.00,0.00
57090,0.0,tcp,http_443,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,12.0,0.05,0.05,0.00,0.00,1.0,1.0,0.00,0.00
59386,0.0,tcp,http,SF,338.0,1158.0,0,0.0,0.0,0.0,...,29.0,255.0,1.00,0.00,0.03,0.05,0.0,0.0,0.00,0.00
29823,0.0,tcp,whois,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,3.0,0.01,0.05,0.00,0.00,1.0,1.0,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50433,0.0,tcp,http,SF,313.0,1086.0,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.00,0.00
18596,0.0,tcp,http,SF,333.0,864.0,0,0.0,0.0,0.0,...,79.0,255.0,1.00,0.00,0.01,0.07,0.0,0.0,0.78,0.85
24293,0.0,udp,private,SF,1.0,0.0,0,0.0,0.0,0.0,...,255.0,19.0,0.07,0.23,1.00,0.00,0.0,0.0,0.00,0.00
64492,0.0,tcp,http,SF,210.0,299.0,0,0.0,0.0,0.0,...,29.0,255.0,1.00,0.00,0.03,0.01,0.0,0.0,0.00,0.00


In [48]:
class CustomScaler(BaseEstimator, TransformerMixin):
    def __init__(self, attributes):
        self.attributes = attributes
    def fit(self, X, y=None): #no hace nada
        return self # nothing else to do
    def transform(self, X, y=None): # recibe un dataframe de pandas, recibiendo un conjunto de atributos que queremos escalar
        X_copy = X.copy()
        scale_attrs = X_copy[self.attributes]
        robust_scaler = RobustScaler()
        X_scaled = robust_scaler.fit_transform(scale_attrs)
        X_scaled = pd.DataFrame(X_scaled,columns=self.attributes, index=X_copy.index) #lo transformo en un dataframe
        for attr in self.attributes:
                X_copy[attr] = X_scaled[attr]
        return X_copy

In [50]:
escala = CustomScaler(['src_bytes', 'dst_bytes']).fit_transform(X_train_prep)

In [51]:
X_train_prep.head(10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
23846,0.0,udp,domain_u,SF,46.0,46.0,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
29862,0.0,tcp,sql_net,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,5.0,0.02,0.07,0.00,0.00,1.0,1.0,0.0,0.0
57090,0.0,tcp,http_443,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,12.0,0.05,0.05,0.00,0.00,1.0,1.0,0.0,0.0
59386,0.0,tcp,http,SF,338.0,1158.0,0,0.0,0.0,0.0,...,29.0,255.0,1.00,0.00,0.03,0.05,0.0,0.0,0.0,0.0
29823,0.0,tcp,whois,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,3.0,0.01,0.05,0.00,0.00,1.0,1.0,0.0,0.0
36849,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,20.0,0.08,0.06,0.00,0.00,1.0,1.0,0.0,0.0
22383,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,5.0,0.02,0.07,0.00,0.00,1.0,1.0,0.0,0.0
65402,0.0,tcp,http,SF,340.0,750.0,0,0.0,0.0,0.0,...,102.0,255.0,1.00,0.00,0.01,0.04,0.0,0.0,0.0,0.0
24297,0.0,tcp,http,SF,208.0,706.0,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
6928,13.0,tcp,smtp,SF,1676.0,366.0,0,0.0,0.0,0.0,...,179.0,110.0,0.61,0.02,0.01,0.00,0.0,0.0,0.0,0.0


In [53]:
class CustomOneHotEncoding(BaseEstimator, TransformerMixin):
    def __init__(self):
        self._oh = OneHotEncoder(sparse=False)
        self._columns = None
    def fit(self, X, y=None):
        X_cat = X.select_dtypes(include=['object'])
        self._columns = pd.get_dummies(X_cat).columns
        self._oh.fit(X_cat) # devolvemos las variables categóricas en numéricas
        return self
    def transform(self, X, y=None):
        X_copy = X.copy()
        X_cat = X_copy.select_dtypes(include=['object'])
        X_num = X_copy.select_dtypes(exclude=['object'])
        X_cat_oh = self._oh.transform(X_cat)
        X_cat_oh = pd.DataFrame(X_cat_oh,
        columns=self._columns,
        index=X_copy.index)
        X_copy.drop(list(X_cat), axis=1, inplace=True)
        return X_copy.join(X_cat_oh)

In [54]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler #transformador RobusScaler que me escala las variables numericas
from sklearn.impute import SimpleImputer # transfomador que sustituye los NaN por otro valor que yo le indique

In [57]:
#le paso en primer lugar el transformador NaN con la mediana y este output será escalado por RobustScaler como entrada
num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")),
('rbst_scaler', RobustScaler()), #y el segundo escala solo mis variables
])

In [58]:
X_train_num = X_train.select_dtypes(exclude=['object'])
X_train_prep = num_pipeline.fit_transform(X_train_num) # con las características solo numéricas para todos los transformadores
X_train_prep = pd.DataFrame(X_train_prep,
columns=X_train_num.columns, index=X_train_num.index)

In [60]:
 X_train_num.head(10)

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
23846,0.0,46.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
29862,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,255.0,5.0,0.02,0.07,0.00,0.00,1.0,1.0,0.0,0.0
57090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,255.0,12.0,0.05,0.05,0.00,0.00,1.0,1.0,0.0,0.0
59386,0.0,338.0,1158.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,29.0,255.0,1.00,0.00,0.03,0.05,0.0,0.0,0.0,0.0
29823,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,255.0,3.0,0.01,0.05,0.00,0.00,1.0,1.0,0.0,0.0
36849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,255.0,20.0,0.08,0.06,0.00,0.00,1.0,1.0,0.0,0.0
22383,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,255.0,5.0,0.02,0.07,0.00,0.00,1.0,1.0,0.0,0.0
65402,0.0,340.0,750.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,102.0,255.0,1.00,0.00,0.01,0.04,0.0,0.0,0.0,0.0
24297,0.0,208.0,706.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
6928,13.0,1676.0,366.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,179.0,110.0,0.61,0.02,0.01,0.00,0.0,0.0,0.0,0.0


In [61]:
X_train_prep.head(10)

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
23846,0.0,0.007326,0.093496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.787755,0.526316,-0.428571,0.000000,0.0,0.0,0.0,0.0,0.0
29862,0.0,-0.161172,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,-0.232653,-0.505263,0.571429,0.000000,0.0,1.0,1.0,0.0,0.0
57090,0.0,-0.161172,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,-0.204082,-0.473684,0.285714,0.000000,0.0,1.0,1.0,0.0,0.0
59386,0.0,1.076923,2.353659,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.321637,0.787755,0.526316,-0.428571,0.500000,2.5,0.0,0.0,0.0,0.0
29823,0.0,-0.161172,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,-0.240816,-0.515789,0.285714,0.000000,0.0,1.0,1.0,0.0,0.0
36849,0.0,-0.161172,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,-0.171429,-0.442105,0.428571,0.000000,0.0,1.0,1.0,0.0,0.0
22383,0.0,-0.161172,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,-0.232653,-0.505263,0.571429,0.000000,0.0,1.0,1.0,0.0,0.0
65402,0.0,1.084249,1.524390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.894737,0.787755,0.526316,-0.428571,0.166667,2.0,0.0,0.0,0.0,0.0
24297,0.0,0.600733,1.434959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.787755,0.526316,-0.428571,0.000000,0.0,0.0,0.0,0.0,0.0
6928,13.0,5.978022,0.743902,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.444444,0.195918,0.115789,-0.142857,0.166667,0.0,0.0,0.0,0.0,0.0


In [62]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [63]:
num_attribs = list(X_train.select_dtypes(exclude=['object']))
#solo atributos numericos
cat_attribs = list(X_train.select_dtypes(include=['object']))
#solo atributos categoricos

In [64]:
full_pipeline = ColumnTransformer([
("num", num_pipeline, num_attribs),
("cat", OneHotEncoder(), cat_attribs),
])

In [65]:
X_train_prep = CustomScaler(['src_bytes', 'dst_bytes']).fit_transform(X_train)

In [66]:
X_train_prep = pd.DataFrame(X_train_prep,
columns=list(pd.get_dummies(X_train)), index=X_train.index)

In [67]:
X_train_prep.head(10)

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,flag_S3,flag_SF,flag_SH,land_0,land_1,logged_in_0,logged_in_1,is_host_login_0,is_guest_login_0,is_guest_login_1
23846,0.0,0.007326,0.093496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29862,0.0,-0.161172,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57090,0.0,-0.161172,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59386,0.0,1.076923,2.353659,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29823,0.0,-0.161172,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36849,0.0,-0.161172,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22383,0.0,-0.161172,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65402,0.0,1.084249,1.524390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24297,0.0,0.600733,1.434959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6928,13.0,5.978022,0.743902,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
X_train.head(10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
23846,0.0,udp,domain_u,SF,46.0,46.0,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
29862,0.0,tcp,sql_net,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,5.0,0.02,0.07,0.00,0.00,1.0,1.0,0.0,0.0
57090,0.0,tcp,http_443,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,12.0,0.05,0.05,0.00,0.00,1.0,1.0,0.0,0.0
59386,0.0,tcp,http,SF,338.0,1158.0,0,0.0,0.0,0.0,...,29.0,255.0,1.00,0.00,0.03,0.05,0.0,0.0,0.0,0.0
29823,0.0,tcp,whois,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,3.0,0.01,0.05,0.00,0.00,1.0,1.0,0.0,0.0
36849,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,20.0,0.08,0.06,0.00,0.00,1.0,1.0,0.0,0.0
22383,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,5.0,0.02,0.07,0.00,0.00,1.0,1.0,0.0,0.0
65402,0.0,tcp,http,SF,340.0,750.0,0,0.0,0.0,0.0,...,102.0,255.0,1.00,0.00,0.01,0.04,0.0,0.0,0.0,0.0
24297,0.0,tcp,http,SF,208.0,706.0,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
6928,13.0,tcp,smtp,SF,1676.0,366.0,0,0.0,0.0,0.0,...,179.0,110.0,0.61,0.02,0.01,0.00,0.0,0.0,0.0,0.0
